<a href="https://colab.research.google.com/github/Yyzhang2000/AI-Cookbook/blob/main/llm_agent_function_calling_%E2%9C%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

In [2]:
from openai import OpenAI
import requests
import json

In [3]:
OPENAI_API_KEY="YOUR-OPENAI-API-KEY"

In [4]:
# 1.define get weather function
def get_current_weather(location: str, unit: str = "celsius") -> dict:
    fake_weather_db = {
        "Singapore": {
            "celsius": {"temperature": 30, "weather": "humid and cloudy"},
            "fahrenheit": {"temperature": 86, "weather": "humid and cloudy"}
        },
        "New York": {
            "celsius": {"temperature": 22, "weather": "sunny"},
            "fahrenheit": {"temperature": 72, "weather": "sunny"}
        },
        "London": {
            "celsius": {"temperature": 15, "weather": "rainy"},
            "fahrenheit": {"temperature": 59, "weather": "rainy"}
        }
    }

    # 默认值
    default_weather = {"temperature": "N/A", "weather": "unknown"}

    location_weather = fake_weather_db.get(location, {})
    weather = location_weather.get(unit, default_weather)

    return {
        "location": location,
        "unit": unit,
        "temperature": weather["temperature"],
        "weather": weather["weather"]
    }

In [5]:
# 2. define tool schema（function calling）
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and country, e.g. Singapore",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit",
                    },
                },
                "required": ["location"],
            },
        }
    }
]

In [6]:
class WeatherAgent:
    def __init__(self, use_tool=True):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.messages = []
        self.use_tool = use_tool

        # tool schema for function calling
        self.tools = tools


    def chat(self, user_input: str) -> str:
        self.messages.append({"role": "user", "content": user_input})

        # 🧠 模型初步生成响应
        if self.use_tool:
            response = self.client.chat.completions.create(
                model="gpt-4-1106-preview",
                messages=self.messages,
                tools=self.tools,
                tool_choice="auto"
            )
        else:
            response = self.client.chat.completions.create(
                model="gpt-4-1106-preview",
                messages=self.messages
            )

        message = response.choices[0].message
        tool_calls = message.tool_calls if self.use_tool else None

        # ✅ 如果模型调用了函数
        if tool_calls:
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                args = json.loads(tool_call.function.arguments)

                if function_name == "get_current_weather":
                    result = get_current_weather(**args)

                    # 把工具调用及返回添加进对话历史
                    self.messages.append(message)
                    self.messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "name": function_name,
                        "content": json.dumps(result)
                    })

                    # 再次生成最终回答
                    final_response = self.client.chat.completions.create(
                        model="gpt-4-1106-preview",
                        messages=self.messages
                    )

                    reply = final_response.choices[0].message.content
                    self.messages.append({"role": "assistant", "content": reply})
                    return "🤖: " + reply
        else:
            # 📦 没有使用工具，直接回复
            reply = message.content
            self.messages.append({"role": "assistant", "content": reply})
            return "🤖: " + reply

In [7]:
agent = WeatherAgent(use_tool=True)
reply = agent.chat("What's the weather in Singapore in fahrenheit?")
print(reply)

🤖: The current weather in Singapore is humid and cloudy with a temperature of 86°F.


In [8]:
agent_no_tool = WeatherAgent(use_tool=False)
reply = agent_no_tool.chat("What's the weather in Singapore in fahrenheit?")
print(reply)

🤖: I'm sorry, but I cannot provide real-time information. However, I can tell you that Singapore has a tropical rainforest climate with no distinct seasons, and its temperature typically ranges from about 75°F to around 89°F (24°C to 32°C) throughout the year. Humidity is high, and rainfall is fairly evenly distributed throughout the year. To get the current weather conditions in Singapore in Fahrenheit, you would need to check a recent weather report or use a weather forecasting service.
